## Prepare Callbacks

In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Bijay\\Downloads\\DJANGO_PROJECTS\\ML\\dl_project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Bijay\\Downloads\\DJANGO_PROJECTS\\ML\\dl_project'

## Entity

In [5]:
from pathlib import Path
from dataclasses import dataclass

# Data are immutable with frozen = true
@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

## Configuration Manager

In [28]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_dir

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAM_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_dir([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        
        config = self.config.prepare_callback_model
        model_ckpt_model = os.path.dirname(config.checkpoint_model_filepath)
        create_dir([
            Path(model_ckpt_model),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir= Path(config.root_dir),
            tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

## Prepare Components

In [25]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [30]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property # with it we can access the method like a variable
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_log_dir_at {timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )

    def get_tb_chpt_callback(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

## Pipeline

In [31]:
try:
    config = ConfigurationManager()
    prepare_callback_config = config.get_prepare_callback_config()
    prepare_callback = PrepareCallback(config=prepare_callback_config)
    callback_list = prepare_callback.get_tb_chpt_callback()
    
except Exception as e:
    raise e

[2025-09-14 15:09:12,659: INFO: common_cmd: Yaml file : config\config.yaml loaded successfully]
[2025-09-14 15:09:12,663: INFO: common_cmd: Yaml file : params.yaml loaded successfully]
[2025-09-14 15:09:12,664: INFO: common_cmd: Directory created succesfully at artifacts]
[2025-09-14 15:09:12,666: INFO: common_cmd: Directory created succesfully at artifacts\prepare_callback\checkpoint_dir]
[2025-09-14 15:09:12,668: INFO: common_cmd: Directory created succesfully at artifacts\prepare_callback\tensorboard_log_dir]
